# Train Mountain Car

[OpenAI Gym](http://gym.openai.com) has been designed in such a way that all environments provide the same API - i.e. the same methods `reset`, `step` and `render`, and the same abstractions of **action space** and **observation space**. Thus is should be possible to adapt the same reinforcement learning algorithms to different environments with minimal code changes.

## A Mountain Car Environment

[Mountain Car environment](https://gym.openai.com/envs/MountainCar-v0/) contains a car stuck in a valley:

The goal is to get out of the valley and capture the flag, by doing at each step one of the following actions:

| Value | Meaning |
|---|---|
| 0 | Accelerate to the left |
| 1 | Do not accelerate |
| 2 | Accelerate to the right |

The main trick of this problem is, however, that the car's engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum.

Observation space consists of just two values:

| Num | Observation  | Min | Max |
|-----|--------------|-----|-----|
|  0  | Car Position | -1.2| 0.6 |
|  1  | Car Velocity | -0.07 | 0.07 |

Reward system for the mountain car is rather tricky:

 * Reward of 0 is awarded if the agent reached the flag (position = 0.5) on top of the mountain.
 * Reward of -1 is awarded if the position of the agent is less than 0.5.

Episode terminates if the car position is more than 0.5, or episode length is greater than 200.
## Instructions

Adapt our reinforcement learning algorithm to solve the mountain car problem. Start with existing [notebook.ipynb](notebook.ipynb) code, substitute new environment, change state discretization functions, and try to make existing algorithm to train with minimal code modifications. Optimize the result by adjusting hyperparameters.

> **Note**: Hyperparameters adjustment is likely to be needed to make algorithm converge. 


Solution

In [1]:
import sys
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import random

Create a mountain car environment

In [34]:
env = gym.make("MountainCar-v0",render_mode = "human")
print(env.action_space)
print(env.observation_space)
print(env.action_space.sample())

Discrete(3)
Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
2


To see how the environment works, let's run a short simulation for 100 steps. At each step, we provide one of the actions to be taken - in this simulation we just randomly select an action from action_space.

In [35]:
env = gym.make("MountainCar-v0",render_mode = "human")
observation, info = env.reset()

for i in range(100):
   action = env.action_space.sample()  # agent policy that uses the observation and info
   observation, reward, terminated, truncated, info = env.step(action)
   if terminated or truncated:
      observation, info = env.reset()
env.close()

During simulation, we need to get observations in order to decide how to act. In fact, step function returns us back current observations, reward function, and the done flag that indicates whether it makes sense to continue the simulation or not:

In [36]:
env = gym.make("MountainCar-v0", render_mode = "human")
observation, info = env.reset()

for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"{observation} -> {reward}")

    if terminated or truncated:
        observation, info = env.reset()

env.close()

[-4.3632162e-01  3.5545020e-04] -> -1.0
[-0.43561327  0.00070833] -> -1.0
[-0.4365572  -0.00094393] -> -1.0
[-0.43914655 -0.00258935] -> -1.0
[-0.44236255 -0.00321599] -> -1.0
[-0.4471818  -0.00481924] -> -1.0
[-0.45156914 -0.00438736] -> -1.0
[-0.45749253 -0.00592339] -> -1.0
[-0.46490848 -0.00741595] -> -1.0
[-0.47176233 -0.00685385] -> -1.0
[-0.4780034  -0.00624106] -> -1.0
[-0.48558536 -0.00758196] -> -1.0
[-0.49245182 -0.00686644] -> -1.0
[-0.49855152 -0.00609971] -> -1.0
[-0.5058389  -0.00728738] -> -1.0
[-0.5142594  -0.00842052] -> -1.0
[-0.52374995 -0.00949056] -> -1.0
[-0.53323936 -0.00948942] -> -1.0
[-0.54265654 -0.00941713] -> -1.0
[-0.5529308  -0.01027427] -> -1.0
[-0.56398535 -0.01105457] -> -1.0
[-0.5757378  -0.01175241] -> -1.0
[-0.5861007  -0.01036296] -> -1.0
[-0.59499764 -0.00889693] -> -1.0
[-0.60336316 -0.00836553] -> -1.0
[-0.61013615 -0.00677299] -> -1.0
[-0.6172674  -0.00713122] -> -1.0
[-0.6227053  -0.00543792] -> -1.0
[-0.62641084 -0.00370552] -> -1.0
[-0.6283

We can get min and max value of those numbers:

In [37]:
print(env.observation_space.low)
print(env.observation_space.high)

[-1.2  -0.07]
[0.6  0.07]


# Saving result to an animated GIF
If you want to impress your friends, you may want to send them the animated GIF picture of the balancing pole. To do this, we can invoke env.render to produce an image frame, and then save those to animated GIF using PIL library: